## Make ADS damage images

In `clean_ads_polygons.ipynb`, we ingested the ADS polygon dataset into Earth Engine. Now, we need to convert these to annual images so that we can add them to our TFRecordDataset.

In [1]:
import ee
import geemap

ee.Initialize()

import os
import datetime

if "notebooks" in os.getcwd():
    os.chdir("..")
    print("Changed working dir to", os.getcwd())

Changed working dir to G:\Other computers\My Laptop\UW\Classes\ESS521\project


## Part 1: damage polygons

Get the asset, filter to an arbitrary year, and map.

In [2]:
# Get assets
ads_damage = ee.FeatureCollection("projects/forest-lst/assets/damage_polygons")
ads_survey = ee.FeatureCollection("projects/forest-lst/assets/survey_polygons")

mod_lc = ee.ImageCollection("MODIS/061/MCD12Q1")

# Use California borders as export region
ca = ee.FeatureCollection("TIGER/2018/States")\
    .filter(ee.Filter.eq("NAME", "California"))\
    .first()

In [3]:
damage_years = ads_damage.aggregate_histogram("SURVEY_YEA").getInfo().keys()
survey_years = ads_damage.aggregate_histogram("SURVEY_YEA").getInfo().keys()

print("Damage:", damage_years)
print("Survey:", survey_years)

Damage: dict_keys(['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])
Survey: dict_keys(['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])


In [4]:
# Examine a year of data. How did the trees die?
ads_damage_18 = ads_damage.filter(ee.Filter.equals("SURVEY_YEA", 2018))
print(ads_damage_18.aggregate_histogram("DCA_COMMON").getInfo())

{'California flatheaded borer': 46, 'Jeffrey pine beetle': 618, 'drought': 31, 'fir engraver': 5470, 'flatheaded fir borer': 525, 'goldspotted oak borer': 43, 'mountain pine beetle': 296, 'pinyon ips': 13, 'unknown bark beetle': 14, 'unknown wood borer': 7, 'western pine beetle': 833}


In [5]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.addLayer(ads_damage_18, {}, 'Damage polygons')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Rasterize to 2.5' cells, color pixels by percent of pixel area where mortality occurred.

In [11]:
target_proj = ee.Projection("EPSG:3857")
cell_size = 4000 # m

zero = ee.Image(0)

test_year = 2015

# initial rasterization at 100 m resolution
ads_18_raster = ads_damage_18.map(lambda x: x.set("const", 1))\
    .reduceToImage(["const"], ee.Reducer.max())\
    .rename("mort_pct")

# Blend with the zero image, then reduce resolution to calculate pixel fraction
ads_18_pct_mort = zero.blend(ads_18_raster)\
    .setDefaultProjection(target_proj, None, 50)\
    .reduceResolution(ee.Reducer.mean(), maxPixels=4096, bestEffort=True)\
    .reproject(target_proj, None, cell_size)

# Build the mask - surveyed region that was classified as forest in the 
# annual MODIS LC product.
ads_18_survey = ads_survey.filter(ee.Filter.equals("SURVEY_YEA", test_year))\
    .map(lambda x: x.set("const", 1))

mod_18 = mod_lc.filter(ee.Filter.calendarRange(test_year, test_year, "year")).first()

mask = ads_18_survey.reduceToImage(["const"], ee.Reducer.max())\
    .setDefaultProjection(target_proj)\
    .eq(1)
    #.And(mod_18.select("LC_Type1").lt(6))


ads_18_masked = ads_18_pct_mort.updateMask(mask).reproject(target_proj, None, cell_size)

In [12]:
# Verify that it worked
img_vis = {
    "min": 0,
    "max": 1,
    "palette": ["white", "red"]
}

Map = geemap.Map()
Map.add_basemap("HYBRID")
# Map.addLayer(ads_18_pct_mort, img_vis, "Damage raster")
Map.addLayer(ads_18_masked, img_vis, "Masked damage raster")
Map.addLayer(ads_damage_18, {}, 'Damage polygons')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Make an image for each year, export it to an image collection.

In [15]:
def make_ads_damage_image(year, out_collection, as_img=False):
    # Filter to the given year
    year_filter = ee.Filter.calendarRange(year, year, "year")
    eq_filter   = ee.Filter.eq("SURVEY_YEA", year)
    
    this_damage = ads_damage.filter(eq_filter)
    this_survey = ads_survey.filter(eq_filter)
    
    # Rasterize the damage polygons
    zero = ee.Image(0)
    damage_raster = this_damage.map(lambda x: x.set("const", 1))\
        .reduceToImage(["const"], ee.Reducer.max())

    damage_pct_mort = zero.blend(damage_raster)\
        .setDefaultProjection(target_proj, None, 10)\
        .reduceResolution(ee.Reducer.mean(), maxPixels=4096, bestEffort=True)

    # Make the mask
    mask = this_survey.map(lambda x: x.set("const", 1))\
        .reduceToImage(["const"], ee.Reducer.max())\
        .setDefaultProjection(target_proj)
    
    # Apply the mask and set output projection
    damage_masked = damage_pct_mort.updateMask(mask)\
        .reproject(target_proj, None, cell_size)\
        .rename("pct_mortality")
    
    # Set timekeeping properties
    epoch_start = datetime.datetime(year, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc)
    epoch_end   = datetime.datetime(year+1, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc) - datetime.timedelta(milliseconds=1)

    damage_masked = damage_masked.set({
        "system:time_start": epoch_start.timestamp() * 1000,
        "system:time_end": epoch_end.timestamp() * 1000
    })
    
    # Return export task
    fname = "ads_damage_"+str(year)
    asset = "/".join([out_collection, fname])

    if as_img:
        return damage_masked
    else:
        return ee.batch.Export.image.toAsset(
            damage_masked, description=fname, assetId=asset,
            # N.b. you have to provide a geometry
            region=ca.geometry(), scale=cell_size, crs=target_proj
        )

In [18]:
test_img = make_ads_damage_image(2003, "projects/forest-lst/assets/damage_img", as_img=True)

In [19]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.addLayer(test_img, img_vis, "test image")
Map.add_colorbar(img_vis, label="Mortality Pct", layer_name="test image")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
# Create destination image collection
collection = "projects/forest-lst/assets/damage_img"
os.system("earthengine create collection {}".format(collection))

0

In [21]:
years = list(map(int, set(damage_years) & set(survey_years)))
print(years)

[2019, 2008, 2011, 2013, 2018, 2017, 2012, 2003, 2020, 2023, 2021, 2007, 2004, 2014, 2005, 2002, 2006, 2001, 2010, 2009, 1999, 2022, 2000, 1998, 2016, 2015]


In [22]:
tasks = map(lambda x: make_ads_damage_image(x, collection), years)
for t in tasks: 
    print("Starting", t.config["description"])
    t.start()

Starting ads_damage_2019
Starting ads_damage_2008
Starting ads_damage_2011
Starting ads_damage_2013
Starting ads_damage_2018
Starting ads_damage_2017
Starting ads_damage_2012
Starting ads_damage_2003
Starting ads_damage_2020
Starting ads_damage_2023
Starting ads_damage_2021
Starting ads_damage_2007
Starting ads_damage_2004
Starting ads_damage_2014
Starting ads_damage_2005
Starting ads_damage_2002
Starting ads_damage_2006
Starting ads_damage_2001
Starting ads_damage_2010
Starting ads_damage_2009
Starting ads_damage_1999
Starting ads_damage_2022
Starting ads_damage_2000
Starting ads_damage_1998
Starting ads_damage_2016
Starting ads_damage_2015
